## 1.0 Dependency loading

In [22]:
# Import necessary libraries
import pandas as pd
import numpy as np

## 2.0 Data extraction

In [2]:
# Read the data into a Pandas DataFrame

try:
    # Read the data into a Pandas DataFrame
    raw_data_df = pd.read_excel('data/raw_data.xlsx')
    print("Data loaded successfully.\n")
except FileNotFoundError:
    print("Error: The file 'data/raw_data.xlsx' was not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Data loaded successfully.



In [541]:
# Get a brief summary of the raw_data DataFrame.
raw_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157095 entries, 0 to 157094
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   setting               157095 non-null  object 
 1   date                  157095 non-null  int64  
 2   source                157095 non-null  object 
 3   indicator_abbr        157095 non-null  object 
 4   indicator_name        157095 non-null  object 
 5   dimension             157095 non-null  object 
 6   subgroup              157095 non-null  object 
 7   estimate              153334 non-null  float64
 8   se                    136679 non-null  float64
 9   ci_lb                 142754 non-null  float64
 10  ci_ub                 142754 non-null  float64
 11  population            147089 non-null  float64
 12  flag                  157095 non-null  object 
 13  setting_average       157095 non-null  float64
 14  iso3                  157095 non-null  object 
 15  

There are 157,095 records and 24 columns of raw data in our dataframe.

## 3.0 Data cleaning

### 3.1 Check for any columns that are empty and drop them from the dataframe.

In [4]:
# Check for empty columns
empty_columns = raw_data_df.columns[raw_data_df.isnull().all()]
print("Empty columns:", empty_columns)

# Remove empty columns
cleaned_df = raw_data_df.dropna(axis=1, how='all')

# Verify data frame info
print("DataFrame info after removing empty columns:")
cleaned_df.info()

Empty columns: Index([], dtype='object')
DataFrame info after removing empty columns:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157095 entries, 0 to 157094
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   setting               157095 non-null  object 
 1   date                  157095 non-null  int64  
 2   source                157095 non-null  object 
 3   indicator_abbr        157095 non-null  object 
 4   indicator_name        157095 non-null  object 
 5   dimension             157095 non-null  object 
 6   subgroup              157095 non-null  object 
 7   estimate              153334 non-null  float64
 8   se                    136679 non-null  float64
 9   ci_lb                 142754 non-null  float64
 10  ci_ub                 142754 non-null  float64
 11  population            147089 non-null  float64
 12  flag                  157095 non-null  object 
 13  setting_average   

There are still 24 columns, so no empty columns were found.

### 3.2 Count the unique values in each column to check for single values, we can drop them from the dataframe.  Such columns will not provide any meaningful insights.

In [543]:
# Count the number of unique values in each column
unique_counts = cleaned_df.nunique()

# Create a new DataFrame to store the counts
unique_counts_df = pd.DataFrame(unique_counts, columns=['Unique Count'])

# Display the new DataFrame
print("DataFrame with the count of unique values in each column:")
unique_counts_df

DataFrame with the count of unique values in each column:


,Unique Count
setting,156
date,34
source,19
indicator_abbr,15
indicator_name,15
dimension,6
subgroup,4517
estimate,130841
se,135364
ci_lb,136109


Columns 'favourable indicator', 'indicator_scale', 'dataset_id', and 'update' have single values so we can drop them.

In [5]:
# Remove columns with a single value from the DataFrame
cleaned_df = cleaned_df.drop(columns=['favourable_indicator', 'indicator_scale', 'dataset_id', 'update'])

# Display the updated DataFrame
print("DataFrame info after removing single value columns:\n")
cleaned_df.info()

DataFrame info after removing single value columns:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157095 entries, 0 to 157094
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   setting             157095 non-null  object 
 1   date                157095 non-null  int64  
 2   source              157095 non-null  object 
 3   indicator_abbr      157095 non-null  object 
 4   indicator_name      157095 non-null  object 
 5   dimension           157095 non-null  object 
 6   subgroup            157095 non-null  object 
 7   estimate            153334 non-null  float64
 8   se                  136679 non-null  float64
 9   ci_lb               142754 non-null  float64
 10  ci_ub               142754 non-null  float64
 11  population          147089 non-null  float64
 12  flag                157095 non-null  object 
 13  setting_average     157095 non-null  float64
 14  iso3                157095 non-

The four single value columns were correctly dropped.  We now have 20 columns in the dataframe.

### 3.3 Check remaining columns for relevance.

1. The *flag* column includes notes, author, and reference title.
2. The *source* column includes a code for reprenting the source data type.
3. The meaning of the *reference_subgroup* column is unknown.
4. The *se* column refers to the standard error in the prevalence estimate.
5. The *ci_lb* and *ci_ub* columns refer to confidence intervals upper and lower bounds respectively.
6. The *ordered_dimension* column is a boolean flag that indicates if the *dimension* column is a nominal or ordinal variable.  In this dataset, *0* represents a nominal variable and *1* represents an ordinal variable.  Nominal variables are for the dimensions of *sex*, *place of residence*, and *subnational region*.  Ordinal variables are for the dimensions of *age*, *economic status*, and *education level*. Ordinal dimensions use the *subgroup_order* column to denote the specific ordering.
7. The *wbincome2024* column is an income group classification provided by The World Bank.  It is a classification of the subnational region dimension.  This is extra data since economic status is already a covered dimension.
8. The *indicator_name* column is a description for *indicator_abbr*.

None data is needed for our analysis, so we will drop it from the dataframe.

In [545]:
# Remove 'flag' column from the DataFrame
cleaned_df = cleaned_df.drop(columns=['flag', 'source', 'reference_subgroup', 'se', 'ci_lb', 'ci_ub', 'ordered_dimension', 'wbincome2024', 'indicator_name'])

# Display the updated DataFrame
print("DataFrame info after removing the unecessary columns:\n")
cleaned_df.info()

DataFrame info after removing the unecessary columns:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157095 entries, 0 to 157094
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   setting          157095 non-null  object 
 1   date             157095 non-null  int64  
 2   indicator_abbr   157095 non-null  object 
 3   dimension        157095 non-null  object 
 4   subgroup         157095 non-null  object 
 5   estimate         153334 non-null  float64
 6   population       147089 non-null  float64
 7   setting_average  157095 non-null  float64
 8   iso3             157095 non-null  object 
 9   subgroup_order   157095 non-null  int64  
 10  whoreg6          157095 non-null  object 
dtypes: float64(3), int64(2), object(6)
memory usage: 13.2+ MB


### 3.4 Rename the columns with more intuitive titles and reorder them.

In [6]:
# Rename the columns
cleaned_df = cleaned_df.rename(columns={
    'setting': 'Country',
    'date': 'Year',
    'indicator_abbr': 'Anthropometric Indicator',
    'dimension': 'Dimension',
    'subgroup': 'Dimension Value',
    'subgroup_order': 'Dimension Value Order',
    'setting_average': 'Country Avg',
    'iso3': 'Country ISO-3 Code',
    'whoreg6': 'Region',
    'estimate': 'Prevalence Estimate %',
    'population': 'Weighted Sample Size'
})

# Reorder the columns
cleaned_df = cleaned_df[
    [
        'Region',
        'Country ISO-3 Code',
        'Country',
        'Year',
        'Dimension',
        'Dimension Value',
        'Dimension Value Order',
        'Anthropometric Indicator',
        'Prevalence Estimate %',
        'Weighted Sample Size',
        'Country Avg'
    ]
]

# Check the DataFrame information
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157095 entries, 0 to 157094
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Region                    157095 non-null  object 
 1   Country ISO-3 Code        157095 non-null  object 
 2   Country                   157095 non-null  object 
 3   Year                      157095 non-null  int64  
 4   Dimension                 157095 non-null  object 
 5   Dimension Value           157095 non-null  object 
 6   Dimension Value Order     157095 non-null  int64  
 7   Anthropometric Indicator  157095 non-null  object 
 8   Prevalence Estimate %     153334 non-null  float64
 9   Weighted Sample Size      147089 non-null  float64
 10  Country Avg               157095 non-null  float64
dtypes: float64(3), int64(2), object(6)
memory usage: 13.2+ MB


### 3.5 From the summary information, we see that columns Prevalence Estimate % and Weighted Sample Size have empty values.  Those rows need to be removed from the dataframe.

In [7]:
# Remove rows where 'Weighted Sample Size' and 'Prevalence Estimate %' are empty
cleaned_df = cleaned_df.dropna(subset=['Weighted Sample Size', 'Prevalence Estimate %'])

# Check the DataFrame after dropping rows
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 147089 entries, 1 to 157094
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Region                    147089 non-null  object 
 1   Country ISO-3 Code        147089 non-null  object 
 2   Country                   147089 non-null  object 
 3   Year                      147089 non-null  int64  
 4   Dimension                 147089 non-null  object 
 5   Dimension Value           147089 non-null  object 
 6   Dimension Value Order     147089 non-null  int64  
 7   Anthropometric Indicator  147089 non-null  object 
 8   Prevalence Estimate %     147089 non-null  float64
 9   Weighted Sample Size      147089 non-null  float64
 10  Country Avg               147089 non-null  float64
dtypes: float64(3), int64(2), object(6)
memory usage: 13.5+ MB


### 3.6 The Child's Age dimension includes additional indicators that append _M and _F to the Anthropometric Indicator.  These entries are redundant to our analysis as we already have aggregate data for the dimension of Sex.  We remove these rows.

In [8]:
# Remove rows where 'Anthropometric Indicator' ends with '_F' or '_M'
cleaned_df = cleaned_df[~cleaned_df['Anthropometric Indicator'].str.endswith(('_F', '_M'))]

# Check the DataFrame after dropping rows
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100770 entries, 1 to 157082
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Region                    100770 non-null  object 
 1   Country ISO-3 Code        100770 non-null  object 
 2   Country                   100770 non-null  object 
 3   Year                      100770 non-null  int64  
 4   Dimension                 100770 non-null  object 
 5   Dimension Value           100770 non-null  object 
 6   Dimension Value Order     100770 non-null  int64  
 7   Anthropometric Indicator  100770 non-null  object 
 8   Prevalence Estimate %     100770 non-null  float64
 9   Weighted Sample Size      100770 non-null  float64
 10  Country Avg               100770 non-null  float64
dtypes: float64(3), int64(2), object(6)
memory usage: 9.2+ MB


### 3.7 The Sub-national Region dimension value is irrelevant to our analysis and only tied to the World Bank Income Group column, so those rows will be removed.

In [9]:
# Remove entries where 'Dimension' is 'Subnational Region'
cleaned_df = cleaned_df[cleaned_df['Dimension'] != 'Subnational region']

# Check the DataFrame after dropping rows
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56618 entries, 1 to 157082
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Region                    56618 non-null  object 
 1   Country ISO-3 Code        56618 non-null  object 
 2   Country                   56618 non-null  object 
 3   Year                      56618 non-null  int64  
 4   Dimension                 56618 non-null  object 
 5   Dimension Value           56618 non-null  object 
 6   Dimension Value Order     56618 non-null  int64  
 7   Anthropometric Indicator  56618 non-null  object 
 8   Prevalence Estimate %     56618 non-null  float64
 9   Weighted Sample Size      56618 non-null  float64
 10  Country Avg               56618 non-null  float64
dtypes: float64(3), int64(2), object(6)
memory usage: 5.2+ MB


### 3.8 After removing unecessary data, as well as empty rows, we are left with 56,618 rows and 11 columns.

In [10]:
# Check the clean data
cleaned_df.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Weighted Sample Size,Country Avg
1,Eastern Mediterranean,AFG,Afghanistan,2004,Child's age (6 groups) (0-59m),12-23 months,3,overweight,4.3,189.0,4.6
2,Eastern Mediterranean,AFG,Afghanistan,2004,Child's age (6 groups) (0-59m),24-35 months,4,overweight,3.0,182.0,4.6
3,Eastern Mediterranean,AFG,Afghanistan,2004,Child's age (6 groups) (0-59m),36-47 months,5,overweight,5.6,227.0,4.6
4,Eastern Mediterranean,AFG,Afghanistan,2004,Child's age (6 groups) (0-59m),48-59 months,6,overweight,6.4,241.0,4.6
5,Eastern Mediterranean,AFG,Afghanistan,2004,Child's age (6 groups) (0-59m),6-11 months,2,overweight,1.9,107.0,4.6


### 3.9 Clean up

In [11]:
# Apply proper capitalization to Anthropometric Indicator values
mapping = {
    'overweight': 'Overweight',
    'stunting': 'Stunting',
    'underweight': 'Underweight',
    'wasting': 'Wasting',
    'wastingsev': 'Wasting Severe'
}

# Map the values in the 'Anthropometric Indicator' column
cleaned_df['Anthropometric Indicator'] = cleaned_df['Anthropometric Indicator'].replace(mapping)

# Check the unique values in the 'Anthropometric Indicator' column
unique_values = cleaned_df['Anthropometric Indicator'].unique()
print(unique_values)

['Overweight' 'Stunting' 'Underweight' 'Wasting' 'Wasting Severe']


In [12]:
# Clean up dimension names using a dictionary
dimension_mapping = {
    "Child's age (6 groups) (0-59m)": 'Age (months)',
    "Education (3 groups)": "Education (mother)"
}

# Apply the mapping to the 'Dimension' column
cleaned_df['Dimension'] = cleaned_df['Dimension'].replace(dimension_mapping)

# Check the unique values in the column
unique_values = cleaned_df['Dimension'].unique()
print(unique_values)

['Age (months)' 'Sex' 'Place of residence'
 'Economic status (wealth quintile)' 'Education (mother)']


In [13]:
# Export cleaned_df as CSV file
cleaned_df.to_csv("data/clean_data.csv", index=False)

## 4.0 Data normalization
For each of the attributes and dimensions, we create dataframes by filtering the unique values and creating unique IDs for each.  These dataframes will be loaded into the database as tables for future use.

### 4.1 Attributes

#### Country attribute

In [511]:
# Create a new DataFrame with unique values from 'Country ISO-3 Code' and 'Country' columns
country_df = cleaned_df[['Country ISO-3 Code', 'Country']].drop_duplicates()
# Display the new DataFrame
print(country_df)


       Country ISO-3 Code                         Country
1                     AFG                     Afghanistan
1005                  ALB                         Albania
1555                  DZA                         Algeria
2600                  AGO                          Angola
3164                  ARG                       Argentina
...                   ...                             ...
144521                VNM                        Viet Nam
151646                YEM                           Yemen
153070                ZMB                          Zambia
154420                ZWE                        Zimbabwe
156020                PSE  occupied Palestinian territory

[154 rows x 2 columns]


In [512]:
# Create numpy array for each of the countries
country_ids = np.arange(1, len(country_df) + 1)
print(country_ids)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154]


In [513]:
# Use a list comprehension to add "ctry" to each country_id.
ctry_ids = ["ctry" + str(ctry_id) for ctry_id in country_ids]

print(ctry_ids)

['ctry1', 'ctry2', 'ctry3', 'ctry4', 'ctry5', 'ctry6', 'ctry7', 'ctry8', 'ctry9', 'ctry10', 'ctry11', 'ctry12', 'ctry13', 'ctry14', 'ctry15', 'ctry16', 'ctry17', 'ctry18', 'ctry19', 'ctry20', 'ctry21', 'ctry22', 'ctry23', 'ctry24', 'ctry25', 'ctry26', 'ctry27', 'ctry28', 'ctry29', 'ctry30', 'ctry31', 'ctry32', 'ctry33', 'ctry34', 'ctry35', 'ctry36', 'ctry37', 'ctry38', 'ctry39', 'ctry40', 'ctry41', 'ctry42', 'ctry43', 'ctry44', 'ctry45', 'ctry46', 'ctry47', 'ctry48', 'ctry49', 'ctry50', 'ctry51', 'ctry52', 'ctry53', 'ctry54', 'ctry55', 'ctry56', 'ctry57', 'ctry58', 'ctry59', 'ctry60', 'ctry61', 'ctry62', 'ctry63', 'ctry64', 'ctry65', 'ctry66', 'ctry67', 'ctry68', 'ctry69', 'ctry70', 'ctry71', 'ctry72', 'ctry73', 'ctry74', 'ctry75', 'ctry76', 'ctry77', 'ctry78', 'ctry79', 'ctry80', 'ctry81', 'ctry82', 'ctry83', 'ctry84', 'ctry85', 'ctry86', 'ctry87', 'ctry88', 'ctry89', 'ctry90', 'ctry91', 'ctry92', 'ctry93', 'ctry94', 'ctry95', 'ctry96', 'ctry97', 'ctry98', 'ctry99', 'ctry100', 'ctry10

In [514]:
# Add the ctry_ids list as a new column
country_df['Country ID'] = ctry_ids
print(country_df)

       Country ISO-3 Code                         Country Country ID
1                     AFG                     Afghanistan      ctry1
1005                  ALB                         Albania      ctry2
1555                  DZA                         Algeria      ctry3
2600                  AGO                          Angola      ctry4
3164                  ARG                       Argentina      ctry5
...                   ...                             ...        ...
144521                VNM                        Viet Nam    ctry150
151646                YEM                           Yemen    ctry151
153070                ZMB                          Zambia    ctry152
154420                ZWE                        Zimbabwe    ctry153
156020                PSE  occupied Palestinian territory    ctry154

[154 rows x 3 columns]


In [515]:
# Reorder the columns so that 'Country ID' is first
country_df = country_df[['Country ID', 'Country ISO-3 Code', 'Country']]

# Display the updated DataFrame
country_df

,Country ID,Country ISO-3 Code,Country
1,ctry1,AFG,Afghanistan
1005,ctry2,ALB,Albania
1555,ctry3,DZA,Algeria
2600,ctry4,AGO,Angola
3164,ctry5,ARG,Argentina
...,...,...,...
144521,ctry150,VNM,Viet Nam
151646,ctry151,YEM,Yemen
153070,ctry152,ZMB,Zambia
154420,ctry153,ZWE,Zimbabwe


#### Indicator attribute

In [30]:
# Create a new DataFrame with unique values from 'Anthropometric Indicator' and 'Indicator Description' columns
indicator_df = cleaned_df[['Anthropometric Indicator']].drop_duplicates()

# Display the new DataFrame
print(indicator_df)

   Anthropometric Indicator
1                Overweight
21                 Stunting
41              Underweight
61                  Wasting
81           Wasting Severe


In [518]:
# Create numpy array for each of the indicators
indicator_ids = np.arange(1, len(indicator_df) + 1)
print(indicator_ids)

[1 2 3 4 5]


In [519]:
# Use a list comprehension to add "ind" to each ind_id.
ind_ids = ["ind" + str(ind_id) for ind_id in indicator_ids]

print(ind_ids)

['ind1', 'ind2', 'ind3', 'ind4', 'ind5']


In [520]:
# Add the ind_ids list as a new column
indicator_df['Indicator ID'] = ind_ids

print(indicator_df)

   Anthropometric Indicator Indicator ID
1                Overweight         ind1
21                 Stunting         ind2
41              Underweight         ind3
61                  Wasting         ind4
81           Wasting Severe         ind5


In [521]:
# Reorder the columns so that 'Indicator ID' is first
indicator_df = indicator_df[['Indicator ID', 'Anthropometric Indicator']]

# Display the updated DataFrame
indicator_df

,Indicator ID,Anthropometric Indicator
1,ind1,Overweight
21,ind2,Stunting
41,ind3,Underweight
61,ind4,Wasting
81,ind5,Wasting Severe


#### Region attribute

In [535]:
# Create a new DataFrame with unique values from the 'Region' column
region_df = cleaned_df[['Region']].drop_duplicates()

# Display the new DataFrame
print(region_df)

                     Region
1     Eastern Mediterranean
1005               European
1555                African
3164               Americas
4525        Western Pacific
5308        South-East Asia


In [536]:
# Create numpy array for each of the regions
region_ids = np.arange(1, len(region_df) + 1)
print(region_ids)

[1 2 3 4 5 6]


In [537]:
# Use a list comprehension to add "reg" to each region_ids.
reg_ids = ["reg" + str(reg_id) for reg_id in region_ids]

print(reg_ids)

['reg1', 'reg2', 'reg3', 'reg4', 'reg5', 'reg6']


In [538]:
# Add the reg_ids list as a new column
region_df['Region ID'] = reg_ids

print(region_df)

                     Region Region ID
1     Eastern Mediterranean      reg1
1005               European      reg2
1555                African      reg3
3164               Americas      reg4
4525        Western Pacific      reg5
5308        South-East Asia      reg6


In [539]:
# Reorder the columns so that 'Region ID' is first
region_df = region_df[['Region ID', 'Region']]

# Display the updated DataFrame
region_df

,Region ID,Region
1,reg1,Eastern Mediterranean
1005,reg2,European
1555,reg3,African
3164,reg4,Americas
4525,reg5,Western Pacific
5308,reg6,South-East Asia


### 4.2 Dimensions

#### General

In [523]:
# Create a new DataFrame with unique values from the 'Dimension' column
dimension_df = cleaned_df[['Dimension']].drop_duplicates()

# Display the new DataFrame
print(dimension_df)

                             Dimension
1                         Age (months)
6                                  Sex
106                 Place of residence
386  Economic status (wealth quintile)
391                 Education (mother)


In [524]:
# Create numpy array for each of the Dimensions
dimension_ids = np.arange(1, len(dimension_df) + 1)
print(dimension_ids)

[1 2 3 4 5]


In [525]:
# Use a list comprehension to add "dim" to each dimension_id.
dim_ids = ["dim" + str(dim_id) for dim_id in dimension_ids]

print(dim_ids)

['dim1', 'dim2', 'dim3', 'dim4', 'dim5']


In [526]:
# Add the dimension_ids list as a new column
dimension_df['Dimension ID'] = dim_ids

print(dimension_df)

                             Dimension Dimension ID
1                         Age (months)         dim1
6                                  Sex         dim2
106                 Place of residence         dim3
386  Economic status (wealth quintile)         dim4
391                 Education (mother)         dim5


In [527]:
# Reorder the columns so that 'Dimension ID' is first, then 'Dimension'
dimension_df = dimension_df[['Dimension ID', 'Dimension']]

# Display the updated DataFrame
dimension_df

,Dimension ID,Dimension
1,dim1,Age (months)
6,dim2,Sex
106,dim3,Place of residence
386,dim4,Economic status (wealth quintile)
391,dim5,Education (mother)


#### Age

In [599]:
# Step 1: Filter cleaned_df where Dimension is 'Age (months)'
age_group_df = cleaned_df[cleaned_df['Dimension'] == 'Age (months)']

# Step 2: Remove duplicates based on the 'Dimension Value' column
age_df = age_group_df[['Dimension Value']].drop_duplicates()

# Display the new DataFrame
print(age_df)


    Dimension Value
1      12-23 months
2      24-35 months
3      36-47 months
4      48-59 months
5       6-11 months
100      0-5 months


In [600]:
# Create numpy array for each of the age dimension values
age_ids = np.arange(1, len(age_df) + 1)
print(age_ids)

[1 2 3 4 5 6]


In [601]:
# Use a list comprehension to add "ageval" to each age value ID.
ageval_ids = ["ageval" + str(ageval_id) for ageval_id in age_ids]

print(ageval_ids)

['ageval1', 'ageval2', 'ageval3', 'ageval4', 'ageval5', 'ageval6']


In [602]:
# Add the ageval_ids list as a new column
age_df['Age ID'] = ageval_ids

print(age_df)

    Dimension Value   Age ID
1      12-23 months  ageval1
2      24-35 months  ageval2
3      36-47 months  ageval3
4      48-59 months  ageval4
5       6-11 months  ageval5
100      0-5 months  ageval6


In [604]:
# Reorder the columns so that 'Age ID' is first
age_df = age_df[['Age ID', 'Dimension Value']]

# Display the updated DataFrame
age_df

,Age ID,Dimension Value
1,ageval1,12-23 months
2,ageval2,24-35 months
3,ageval3,36-47 months
4,ageval4,48-59 months
5,ageval5,6-11 months
100,ageval6,0-5 months


#### Sex

In [606]:
# Step 1: Filter cleaned_df where Dimension is 'Sex'
sex_group_df = cleaned_df[cleaned_df['Dimension'] == 'Sex']

# Step 2: Remove duplicates based on the 'Dimension Value' column
sex_df = sex_group_df[['Dimension Value']].drop_duplicates()

# Display the new DataFrame
print(sex_df)


  Dimension Value
6          Female
7            Male


In [610]:
# Create numpy array for each of the dimension values
sex_ids = np.arange(1, len(sex_df) + 1)
print(sex_ids)

[1 2]


In [612]:
# Use a list comprehension to add "sex" to each sex ID.
sex_ids = ["sex" + str(sex_id) for sex_id in sex_ids]

print(sex_ids)

['sex1', 'sex2']


In [613]:
# Add the sex_ids list as a new column
sex_df['Sex ID'] = sex_ids

print(sex_df)

  Dimension Value Sex ID
6          Female   sex1
7            Male   sex2


In [614]:
# Reorder the columns so that 'Sex ID' is first
sex_df = sex_df[['Sex ID', 'Dimension Value']]

# Display the updated DataFrame
sex_df

,Sex ID,Dimension Value
6,sex1,Female
7,sex2,Male


#### Place of residence

In [607]:
# Step 1: Filter cleaned_df where Dimension is 'Place of residence'
residence_group_df = cleaned_df[cleaned_df['Dimension'] == 'Place of residence']

# Step 2: Remove duplicates based on the 'Dimension Value' column
residence_df = residence_group_df[['Dimension Value']].drop_duplicates()

# Display the new DataFrame
print(residence_df)


    Dimension Value
106           Rural
107           Urban


In [616]:
# Create numpy array for each of the dimension values
residence_ids = np.arange(1, len(residence_df) + 1)
print(residence_ids)

[1 2]


In [617]:
# Use a list comprehension to add "res" to each residence ID.
residence_ids = ["res" + str(residence_id) for residence_id in residence_ids]

print(residence_ids)

['res1', 'res2']


In [618]:
# Add the residence_ids list as a new column
residence_df['Residence ID'] = residence_ids

print(residence_df)

    Dimension Value Residence ID
106           Rural         res1
107           Urban         res2


In [619]:
# Reorder the columns so that 'Residence ID' is first
residence_df = residence_df[['Residence ID', 'Dimension Value']]

# Display the updated DataFrame
residence_df

,Residence ID,Dimension Value
106,res1,Rural
107,res2,Urban


#### Economic status

In [608]:
# Step 1: Filter cleaned_df where Dimension is 'Economic status (wealth quintile)'
economic_status_group_df = cleaned_df[cleaned_df['Dimension'] == 'Economic status (wealth quintile)']

# Step 2: Remove duplicates based on the 'Dimension Value' column
economic_status_df = economic_status_group_df[['Dimension Value']].drop_duplicates()

# Display the new DataFrame
print(economic_status_df)


          Dimension Value
386  Quintile 1 (poorest)
387            Quintile 2
388            Quintile 3
389            Quintile 4
390  Quintile 5 (richest)


In [621]:
# Create numpy array for each of the dimension values
economic_status_ids = np.arange(1, len(economic_status_df) + 1)
print(economic_status_ids)

[1 2 3 4 5]


In [622]:
# Use a list comprehension to add "econ" to each economic status ID.
economic_status_ids = ["econ" + str(economic_status_id) for economic_status_id in economic_status_ids]

print(economic_status_ids)

['econ1', 'econ2', 'econ3', 'econ4', 'econ5']


In [623]:
# Add the economic_status_ids list as a new column
economic_status_df['Economic Status ID'] = economic_status_ids

print(economic_status_df)

          Dimension Value Economic Status ID
386  Quintile 1 (poorest)              econ1
387            Quintile 2              econ2
388            Quintile 3              econ3
389            Quintile 4              econ4
390  Quintile 5 (richest)              econ5


In [624]:
# Reorder the columns so that 'ID' is first
economic_status_df = economic_status_df[['Economic Status ID', 'Dimension Value']]

# Display the updated DataFrame
economic_status_df

,Economic Status ID,Dimension Value
386,econ1,Quintile 1 (poorest)
387,econ2,Quintile 2
388,econ3,Quintile 3
389,econ4,Quintile 4
390,econ5,Quintile 5 (richest)


#### Education

In [32]:
# Step 1: Filter cleaned_df where Dimension is 'Education (mother)'
education_group_df = cleaned_df[cleaned_df['Dimension'] == 'Education (mother)']

# Step 2: Remove duplicates based on the 'Dimension Value' column
education_df = education_group_df[['Dimension Value']].drop_duplicates()

# Display the new DataFrame
print(education_df)


                   Dimension Value
391                   No education
392              Primary education
393  Secondary or higher education


In [33]:
# Create numpy array for each of the dimension values
education_ids = np.arange(1, len(education_df) + 1)
print(education_ids)

[1 2 3]


In [34]:
# Use a list comprehension to add "edu" to each education ID.
education_ids = ["edu" + str(education_id) for education_id in education_ids]

print(education_ids)

['edu1', 'edu2', 'edu3']


In [35]:
# Add the economic_status_ids list as a new column
education_df['Education ID'] = education_ids

print(education_df)

                   Dimension Value Education ID
391                   No education         edu1
392              Primary education         edu2
393  Secondary or higher education         edu3


In [36]:
# Reorder the columns so that 'ID' is first
education_df = education_df[['Education ID', 'Dimension Value']]

# Display the updated DataFrame
education_df

,Education ID,Dimension Value
391,edu1,No education
392,edu2,Primary education
393,edu3,Secondary or higher education


## 5.0 Data loading
Load dataframes for the cleaned dataset, attributes, and dimensions into the SQLite database.

### 5.1 Establish connection to SQLite database

In [37]:
import sqlite3

try:
    # Establish a SQLite connection
    conn = sqlite3.connect('data/malnutrition_data.db')
    print("Connection to the database established successfully.")
except sqlite3.OperationalError as e:
    print(f"Operational error: {e}")
except sqlite3.DatabaseError as e:
    print(f"Database error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Connection to the database established successfully.


### 5.2 Load clean data

In [15]:
try:
    # Save the cleaned dataset dataframe to the SQLite database
    cleaned_df.to_sql('clean_data', conn, if_exists='replace', index=False)
    print("Data saved to the database successfully.")

    # Query to ensure the data was stored correctly
    df_from_db = pd.read_sql('SELECT * FROM clean_data', conn)
    print("Data retrieved from the database successfully.")
    print(df_from_db.head())
except ValueError as e:
    print(f"Value error: {e}")
except sqlite3.DatabaseError as e:
    print(f"Database error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Data saved to the database successfully.
Data retrieved from the database successfully.
                  Region Country ISO-3 Code      Country  Year     Dimension  \
0  Eastern Mediterranean                AFG  Afghanistan  2004  Age (months)   
1  Eastern Mediterranean                AFG  Afghanistan  2004  Age (months)   
2  Eastern Mediterranean                AFG  Afghanistan  2004  Age (months)   
3  Eastern Mediterranean                AFG  Afghanistan  2004  Age (months)   
4  Eastern Mediterranean                AFG  Afghanistan  2004  Age (months)   

  Dimension Value  Dimension Value Order Anthropometric Indicator  \
0    12-23 months                      3               Overweight   
1    24-35 months                      4               Overweight   
2    36-47 months                      5               Overweight   
3    48-59 months                      6               Overweight   
4     6-11 months                      2               Overweight   

   Prevalence Es

### 5.3 Load attributes

In [660]:
try:
    # Save the country dataframe to the SQLite database
    country_df.to_sql('country', conn, if_exists='replace', index=False)
    print("Data saved to the database successfully.")

    # Query to ensure the data was stored correctly
    df_from_db = pd.read_sql('SELECT * FROM country', conn)
    print("Data retrieved from the database successfully.")
    print(df_from_db.head())
except ValueError as e:
    print(f"Value error: {e}")
except sqlite3.DatabaseError as e:
    print(f"Database error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Data saved to the database successfully.
Data retrieved from the database successfully.
  Country ID Country ISO-3 Code      Country
0      ctry1                AFG  Afghanistan
1      ctry2                ALB      Albania
2      ctry3                DZA      Algeria
3      ctry4                AGO       Angola
4      ctry5                ARG    Argentina


In [661]:
try:
    # Save the indicator dataset dataframe to the SQLite database
    indicator_df.to_sql('indicator', conn, if_exists='replace', index=False)
    print("Data saved to the database successfully.")

    # Query to ensure the data was stored correctly
    df_from_db = pd.read_sql('SELECT * FROM indicator', conn)
    print("Data retrieved from the database successfully.")
    print(df_from_db.head())
except ValueError as e:
    print(f"Value error: {e}")
except sqlite3.DatabaseError as e:
    print(f"Database error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Data saved to the database successfully.
Data retrieved from the database successfully.
  Indicator ID Anthropometric Indicator
0         ind1               Overweight
1         ind2                 Stunting
2         ind3              Underweight
3         ind4                  Wasting
4         ind5           Wasting Severe


In [662]:
try:
    # Save the region dataframe to the SQLite database
    region_df.to_sql('region', conn, if_exists='replace', index=False)
    print("Data saved to the database successfully.")

    # Query to ensure the data was stored correctly
    df_from_db = pd.read_sql('SELECT * FROM region', conn)
    print("Data retrieved from the database successfully.")
    print(df_from_db.head())
except ValueError as e:
    print(f"Value error: {e}")
except sqlite3.DatabaseError as e:
    print(f"Database error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Data saved to the database successfully.
Data retrieved from the database successfully.
  Region ID                 Region
0      reg1  Eastern Mediterranean
1      reg2               European
2      reg3                African
3      reg4               Americas
4      reg5        Western Pacific


### 5.4 Load dimensions

In [663]:
try:
    # Save the dimension dataframe to the SQLite database
    dimension_df.to_sql('dimension', conn, if_exists='replace', index=False)
    print("Data saved to the database successfully.")

    # Query to ensure the data was stored correctly
    df_from_db = pd.read_sql('SELECT * FROM dimension', conn)
    print("Data retrieved from the database successfully.")
    print(df_from_db.head())
except ValueError as e:
    print(f"Value error: {e}")
except sqlite3.DatabaseError as e:
    print(f"Database error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Data saved to the database successfully.
Data retrieved from the database successfully.
  Dimension ID                          Dimension
0         dim1                       Age (months)
1         dim2                                Sex
2         dim3                 Place of residence
3         dim4  Economic status (wealth quintile)
4         dim5                 Education (mother)


In [664]:
try:
    # Save the age dataframe to the SQLite database
    age_df.to_sql('age', conn, if_exists='replace', index=False)
    print("Data saved to the database successfully.")

    # Query to ensure the data was stored correctly
    df_from_db = pd.read_sql('SELECT * FROM age', conn)
    print("Data retrieved from the database successfully.")
    print(df_from_db.head())
except ValueError as e:
    print(f"Value error: {e}")
except sqlite3.DatabaseError as e:
    print(f"Database error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Data saved to the database successfully.
Data retrieved from the database successfully.
    Age ID Dimension Value
0  ageval1    12-23 months
1  ageval2    24-35 months
2  ageval3    36-47 months
3  ageval4    48-59 months
4  ageval5     6-11 months


In [665]:
try:
    # Save the sex dataframe to the SQLite database
    sex_df.to_sql('sex', conn, if_exists='replace', index=False)
    print("Data saved to the database successfully.")

    # Query to ensure the data was stored correctly
    df_from_db = pd.read_sql('SELECT * FROM sex', conn)
    print("Data retrieved from the database successfully.")
    print(df_from_db.head())
except ValueError as e:
    print(f"Value error: {e}")
except sqlite3.DatabaseError as e:
    print(f"Database error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Data saved to the database successfully.
Data retrieved from the database successfully.
  Sex ID Dimension Value
0   sex1          Female
1   sex2            Male


In [666]:
try:
    # Save the residence dataframe to the SQLite database
    residence_df.to_sql('residence', conn, if_exists='replace', index=False)
    print("Data saved to the database successfully.")

    # Query to ensure the data was stored correctly
    df_from_db = pd.read_sql('SELECT * FROM residence', conn)
    print("Data retrieved from the database successfully.")
    print(df_from_db.head())
except ValueError as e:
    print(f"Value error: {e}")
except sqlite3.DatabaseError as e:
    print(f"Database error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Data saved to the database successfully.
Data retrieved from the database successfully.
  Residence ID Dimension Value
0         res1           Rural
1         res2           Urban


In [667]:
try:
    # Save the economic status dataframe to the SQLite database
    economic_status_df.to_sql('economic_status', conn, if_exists='replace', index=False)
    print("Data saved to the database successfully.")

    # Query to ensure the data was stored correctly
    df_from_db = pd.read_sql('SELECT * FROM economic_status', conn)
    print("Data retrieved from the database successfully.")
    print(df_from_db.head())
except ValueError as e:
    print(f"Value error: {e}")
except sqlite3.DatabaseError as e:
    print(f"Database error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Data saved to the database successfully.
Data retrieved from the database successfully.
  Economic Status ID       Dimension Value
0              econ1  Quintile 1 (poorest)
1              econ2            Quintile 2
2              econ3            Quintile 3
3              econ4            Quintile 4
4              econ5  Quintile 5 (richest)


In [40]:
try:
    # Save the education dataframe to the SQLite database
    education_df.to_sql('education', conn, if_exists='replace', index=False)
    print("Data saved to the database successfully.")

    # Query to ensure the data was stored correctly
    df_from_db = pd.read_sql('SELECT * FROM education', conn)
    print("Data retrieved from the database successfully.")
    print(df_from_db.head())
except ValueError as e:
    print(f"Value error: {e}")
except sqlite3.DatabaseError as e:
    print(f"Database error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Data saved to the database successfully.
Data retrieved from the database successfully.
  Education ID                Dimension Value
0         edu1                   No education
1         edu2              Primary education
2         edu3  Secondary or higher education


### 5.5 Close connection to SQLite database

In [41]:
if conn:
    conn.close()
    print("Database connection closed.")

Database connection closed.


## EXAMPLES: Data extraction from SQLite database (DELETE BEFORE SUBMITTING)

In [568]:
# Query the database from the clean data table by the age dimension and create a dataframe
age_df_from_db = pd.read_sql("SELECT * FROM clean_data WHERE Dimension == 'Age (months)'", conn)
age_df_from_db.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Under-Five Population (million),Country Avg
0,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),12-23 months,3,Overweight,4.3,189.0,4.6
1,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),24-35 months,4,Overweight,3.0,182.0,4.6
2,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),36-47 months,5,Overweight,5.6,227.0,4.6
3,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),48-59 months,6,Overweight,6.4,241.0,4.6
4,Eastern Mediterranean,AFG,Afghanistan,2004,Age (months),6-11 months,2,Overweight,1.9,107.0,4.6


In [577]:
# Query the database from the clean data table by age and create a dataframe
economic_status_df_from_db = pd.read_sql("SELECT * FROM clean_data WHERE Dimension == 'Economic status (wealth quintile)'", conn)
economic_status_df_from_db.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Under-Five Population (million),Country Avg
0,Eastern Mediterranean,AFG,Afghanistan,2018,Economic status (wealth quintile),Quintile 1 (poorest),1,Overweight,4.486930,3576.301514,4.112625
1,Eastern Mediterranean,AFG,Afghanistan,2018,Economic status (wealth quintile),Quintile 2,2,Overweight,4.670275,3767.884277,4.112625
2,Eastern Mediterranean,AFG,Afghanistan,2018,Economic status (wealth quintile),Quintile 3,3,Overweight,4.078213,4178.454102,4.112625
3,Eastern Mediterranean,AFG,Afghanistan,2018,Economic status (wealth quintile),Quintile 4,4,Overweight,3.264172,4315.422852,4.112625
4,Eastern Mediterranean,AFG,Afghanistan,2018,Economic status (wealth quintile),Quintile 5 (richest),5,Overweight,4.254687,3575.646240,4.112625


In [578]:
# Query the database from the clean data table by economic status and create a dataframe
education_df_from_db = pd.read_sql("SELECT * FROM clean_data WHERE Dimension == 'Education (mother)'", conn)
education_df_from_db.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Under-Five Population (million),Country Avg
0,Eastern Mediterranean,AFG,Afghanistan,2018,Education (mother),No education,1,Overweight,4.183403,16289.356445,4.112625
1,Eastern Mediterranean,AFG,Afghanistan,2018,Education (mother),Primary education,2,Overweight,2.141115,1341.238159,4.112625
2,Eastern Mediterranean,AFG,Afghanistan,2018,Education (mother),Secondary or higher education,3,Overweight,4.753134,1926.877563,4.112625
3,Eastern Mediterranean,AFG,Afghanistan,2018,Education (mother),No education,1,Stunting,40.559396,15851.871094,38.192776
4,Eastern Mediterranean,AFG,Afghanistan,2018,Education (mother),Primary education,2,Stunting,29.575229,1330.618286,38.192776


In [579]:
# Query the database from the clean data table by place of residence and create a dataframe
residence_df_from_db = pd.read_sql("SELECT * FROM clean_data WHERE Dimension == 'Place of residence'", conn)
residence_df_from_db.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Under-Five Population (million),Country Avg
0,Eastern Mediterranean,AFG,Afghanistan,2013,Place of residence,Rural,0,Overweight,5.969128,3.234018e+06,5.341543
1,Eastern Mediterranean,AFG,Afghanistan,2013,Place of residence,Urban,0,Overweight,3.338518,1.013278e+06,5.341543
2,Eastern Mediterranean,AFG,Afghanistan,2013,Place of residence,Rural,0,Stunting,43.330780,3.214813e+06,40.413544
3,Eastern Mediterranean,AFG,Afghanistan,2013,Place of residence,Urban,0,Stunting,31.130192,1.010235e+06,40.413544
4,Eastern Mediterranean,AFG,Afghanistan,2013,Place of residence,Rural,0,Underweight,26.375696,3.373931e+06,24.643473


In [580]:
# Query the database from the clean data table by sex and create a dataframe
sex_df_from_db = pd.read_sql("SELECT * FROM clean_data WHERE Dimension == 'Sex'", conn)
sex_df_from_db.head()

,Region,Country ISO-3 Code,Country,Year,Dimension,Dimension Value,Dimension Value Order,Anthropometric Indicator,Prevalence Estimate %,Under-Five Population (million),Country Avg
0,Eastern Mediterranean,AFG,Afghanistan,2004,Sex,Female,0,Overweight,2.6,436.0,4.6
1,Eastern Mediterranean,AFG,Afghanistan,2004,Sex,Male,0,Overweight,6.3,510.0,4.6
2,Eastern Mediterranean,AFG,Afghanistan,2004,Sex,Female,0,Stunting,58.5,436.0,59.3
3,Eastern Mediterranean,AFG,Afghanistan,2004,Sex,Male,0,Stunting,60.0,510.0,59.3
4,Eastern Mediterranean,AFG,Afghanistan,2004,Sex,Female,0,Underweight,33.0,436.0,32.9


In [581]:
# Export the dataframes to CSV files
age_df_from_db.to_csv("data/clean_data_by_age_from_db.csv", index=False)
economic_status_df_from_db.to_csv("data/clean_data_by_economic_status_from_db.csv", index=False)
education_df_from_db.to_csv("data/clean_data_by_education_from_db.csv", index=False)
residence_df_from_db.to_csv("data/clean_data_by_residence_from_db.csv", index=False)
sex_df_from_db.to_csv("data/clean_data_by_sex_from_db.csv", index=False)